In [ ]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# Create DataFrame

ETL framework to get daily temperatures for Santa Barbara airport

In [ ]:
df = pd.read_csv("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_station/USW00023190.csv.gz",
                 compression = 'gzip',
                 names = ['station','date','datatype','value','U1','U2','U3','U4'],
                 low_memory = False)

df["date"] = pd.to_datetime(df["date"].astype(str))
df = df.loc[(df["datatype"].isin(["TMIN","TMAX","PRCP"])) & (df["date"] >= "1970-01-01"), ["date", "datatype", "value"]]
df["value"] = np.where(np.array(df["datatype"]) == "PRCP", np.round(np.array(df["value"])/100/2.54, 2), np.round(np.array(df["value"])/10 * 1.8 + 32, 0))
df = df.pivot(index = "date", columns = "datatype", values = "value").dropna().rename_axis(None, axis = 1).reset_index(inplace = False)
df["TAVG"] = (df["TMIN"] + df["TMAX"]) / 2
df = df.rename(columns = {"date": "Date", "TMAX": "High Temperature", "TMIN": "Low Temperature", "TAVG": "Average Temperature", "PRCP": "Rainfall"})

In [ ]:
df = df[["Date","High Temperature","Low Temperature"]]

# KMeans clustering

Create list object to store data to use in clustering algorithm

In [ ]:
data = list(zip(df["Low Temperature"].tolist(), df["High Temperature"].tolist()))

Initialize KMeans

In [ ]:
kmeans = KMeans(n_clusters = 2)

Fit KMeans with data

In [ ]:
kmeans.fit(data)

Determine label for each data point

In [ ]:
kmeans.labels_

Find assign each data point to one of k cluster from KMeans algorithm output

In [ ]:
df = df.assign(cluster = kmeans.labels_)
df.tail(1000)